In [17]:
import argparse
import numpy as np

RATING_FILE_NAME = dict({'movie': 'ratings.dat', 'book': 'BX-Book-Ratings.csv', 'news': 'ratings.txt', "naver":"ratings.dat"})
SEP = dict({'movie': '::', 'book': ';', 'news': '\t', "naver":"::"})
THRESHOLD = dict({'movie': 4, 'book': 0, 'news': 0,"naver":4})

args = argparse.Namespace()
args.dataset="naver"
DATASET = args.dataset
entity_id2index = dict()
relation_id2index = dict()
item_index_old2new = dict()

In [ ]:
np.random.seed(2022)

In [27]:
with open("./ratings.dat", "r") as f:
    ratings = f.readlines()
ratings = [line.replace("\n","") for line in ratings]
ratings = [line.split("::") for line in ratings]
ratings[:5]

[['0', '0', '5'],
 ['0', '18', '5'],
 ['0', '17', '5'],
 ['0', '15', '5'],
 ['0', '14', '5']]

In [41]:
item_set = set()
for line in ratings:
    item_idx = int(line[1])
    item_set.add(item_idx)

In [42]:
len(item_set)

53671

In [43]:
def negative_samplig(user_set, user_pos_ratings):
    neg_items = dict()
    for user_idx in user_set:
        neg_items[user_idx] = set()
    for user_idx in user_set:
        
        while True:
            neg_item = np.random.randint(low=0, high=len(item_set), size=1)[0]
            if neg_item in user_pos_ratings[user_idx]:
                continue
            # print(user_idx, neg_item)
            neg_items[user_idx].add(neg_item)
            if len(neg_items[user_idx]) == len(user_pos_ratings[user_idx]):
                break
    return neg_items



In [44]:


file = './' + RATING_FILE_NAME[DATASET]

print('reading rating file ...')

user_pos_ratings = dict()
user_neg_ratings = dict()

for line in open(file, encoding='utf-8').readlines()[1:]:
    array = line.strip().split(SEP[DATASET])

    # remove prefix and suffix quotation marks for BX dataset
    if DATASET == 'book':
        array = list(map(lambda x: x[1:-1], array))

    # item_index_old = array[1]
    # if item_index_old not in item_index_old2new:  # the item is not in the final item set
    #     continue
    # item_index = item_index_old2new[item_index_old]
    item_index = array[1]
    user_index = int(array[0])

    rating = float(array[2])
    
    if user_index not in user_pos_ratings:
        user_pos_ratings[user_index] = set()
    user_pos_ratings[user_index].add(item_index)


    


reading rating file ...


In [45]:
len(item_set)

53671

In [46]:
user_set = set(user_pos_ratings.keys())    
user_neg_ratings = negative_samplig(user_set, user_pos_ratings)    


In [47]:
user_neg_ratings[1].intersection(user_pos_ratings[1])

set()

In [53]:
len(user_pos_ratings[0])

19

In [50]:

print('converting rating file ...')
writer = open('./ratings_final_temp.txt', 'w', encoding='utf-8')
user_cnt = 0
user_index_old2new = dict()
for user_index_old, pos_item_set in user_pos_ratings.items():
    if user_index_old not in user_index_old2new:
        user_index_old2new[user_index_old] = user_cnt
        user_cnt += 1
    user_index = user_index_old2new[user_index_old]

    for item in pos_item_set:
        writer.write('%d\t%d\t1\n' % (user_index, int(item)))
        
    unwatched_set = item_set - pos_item_set
    if user_index_old in user_neg_ratings:
        unwatched_set -= user_neg_ratings[user_index_old]
    for item in np.random.choice(list(unwatched_set), size=len(pos_item_set), replace=False):
        writer.write('%d\t%d\t0\n' % (user_index, item))
writer.close()
print('number of users: %d' % user_cnt)
print('number of items: %d' % len(item_set))


converting rating file ...
number of users: 190212
number of items: 53671


In [56]:
user_pos_ratings[0]

{'0',
 '1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9'}

In [57]:
user_neg_ratings[0]

{618,
 2269,
 3534,
 8106,
 11095,
 14773,
 16465,
 22403,
 23604,
 28800,
 32638,
 36089,
 38180,
 38253,
 39245,
 43122,
 45614,
 46519,
 48985}